# Introduction to Quantum Safe Cryptography

This notebook serves as a compilation of the hands-on examples provided in the 'Introduction to Quantum Safe Cryptography' provided by IBM.

Requirements:

-   'cryptography' library 

## CHFs: Cryptographic HASH FUNCTIONS

### The SHA-256 algorithm

Let us demonstrate cryptographic hashing my making use of the well-known SHA-256 algorithm, widely used in modern technologies such as the Blockchain (cryptocurrencies), digital signatures and certificates.

A good hash function will produce digests with large differences even for input strings with small variations.

In [1]:
# Begin by importing the necessary modules

from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes

We generate now two very similar messages which we will feed to the hash function to compare the outputs. 

In [2]:
#Helper function that returns the number of characters different in two strings

def char_diff(str1, str2):
    return sum ( str1[i] != str2[i] for i in range(len(str1)) )

# Messages to be hashed

message_1 = b"Buy 10000 shares of WXYZ stock now!"
message_2 = b"Buy 10000 shares of VXYZ stock now!"

print(f"The two messages differ by { char_diff(message_1, message_2)} characters")

The two messages differ by 1 characters


We now have to instantiate a hash object and update it with the message to be hashed. We then finalize the hash and print the digest. This involves the calls to three methods.

In [3]:
# Create new SHA-256 hash objects, one for each message

chf_1 = hashes.Hash(hashes.SHA256(), backend=default_backend())
chf_2 = hashes.Hash(hashes.SHA256(), backend=default_backend())

# Update each hash object with the bytes of the corresponding message

chf_1.update(message_1)
chf_2.update(message_2)

# Finalize the hash process and obtain the digests

digest_1 = chf_1.finalize()
digest_2 = chf_2.finalize()

In [4]:
digest_1

b'n\x0eba\xb7\x13\x1b\xd8\x0f\xfd\xb2\xa4\xd4/\x9d\x04&65\x0eE\xe1\x84\xb9/\xcb\xcc\x96F\xea\xf1\xe7'

We need to convert the digests to some more readable (or printeable) format, e.g. hexadecimal strings.

In [5]:
#Convert the resulting hash to hexadecimal strings for convenient printing

digest_1_str = digest_1.hex()
digest_2_str = digest_2.hex()

#Print out the digests as strings 

print(f"digest-1: {digest_1_str}")
print(f"digest-2: {digest_2_str}")

print(f"The two digests differ by { char_diff(digest_1_str, digest_2_str)} characters")

digest-1: 6e0e6261b7131bd80ffdb2a4d42f9d042636350e45e184b92fcbcc9646eaf1e7
digest-2: 6b0abb368c3a1730f935b68105e3f3ae7fd43d7e786d3ed3503dbb45c74ada46
The two digests differ by 57 characters


This shows the 'avalanche effect' in the SHA-256 algorithm. We started with two strings difering only in one character. The digests then differ by 57 characters!

## SYMMETRIC KEY CRYPTOGRAPHY (SKC)

### Illustration of Symmetric Key Encryption (SKE)

We illustrate the implementation of the encrypt and decrypt operations using the classical cipher Caesar and the more modern Advanced Encryption System  (AES). The latter has been the standard for symmetric key encryption since 2001.

In [6]:
# import the required crypto functions which will be demonstrated later
from secretpy import Caesar

from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from functools import reduce
import numpy as np

In [7]:
# We now write down a message for encryption

plaintext=u"nobody can see this message. in particular, not my wife."

print(f"\nGiven plaintext: {plaintext}")


Given plaintext: nobody can see this message. in particular, not my wife.


#### Caesar shift cipher:

Caesar shift encryption involves defining

 - An alphabet of possible characters to encode
 - A shift value which can be between 0 (unencrypted) and the length of the alphabet. We consider this the key.

It is known as a monoalphabetic substitution cipher since each letter of the plain text is substituted with another in the ciphertext.

In [8]:
# initialize the required python object for doing Caesar shift encryption
caesar_cipher = Caesar()

# Define the shift, ie the key
caesar_key = 5 
print(f"Caesar shift secret key: {caesar_key}")

# Define the alphabet. Note that symbols, special characters and Capital letters must be included in the alphabet.

alphabet=('a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', ' ', '.', ',')
print(f"alphabet: {alphabet}")

Caesar shift secret key: 5
alphabet: ('a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', ' ', '.', ',')


Let's encrypt and check the ciphertext

In [9]:
caeser_ciphertext = caesar_cipher.encrypt(plaintext, caesar_key, alphabet)
print(f"Encrypted caeser shift ciphertext: {caeser_ciphertext}")

Encrypted caeser shift ciphertext: stgtiachfscxjjcymnxcrjxxfljdcnscufwynhzqfwecstycrac.nkjd


Nice! Now, we can decrypt the message. We have to use the same key.

In [10]:
caeser_plaintext = caesar_cipher.decrypt(caeser_ciphertext, caesar_key, alphabet)
print(f"Decrypted caeser shift plaintext: {caeser_plaintext}\n")

Decrypted caeser shift plaintext: nobody can see this message. in particular, not my wife.



Decryption fails if using a different key (shift value)

In [11]:
caesar_key_2 = 10

caeser_plaintext = caesar_cipher.decrypt(caeser_ciphertext, caesar_key_2, alphabet)
print(f"Decrypted caeser shift plaintext: {caeser_plaintext}\n")

Decrypted caeser shift plaintext: ijzj.tv yivn,,vocdnvh,nnyb,wvdivkymod pgymxvijovhtvrda,w



#### Advanced encryption standard (AES) cipher

We now encrypt the plain text using AES, a popular symmetric key encryption algorithm.

We start by creating the key, in this case, a random 16-letter string.

In [12]:
# lamba defines an inline function in this case that takes two values a,b with the resulting expression of a+b
# reduce uses a two-argument function(above), and applies this to all the entries in the list (random alphabet characters) cumulatively

alphabet=('a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', ' ')

aes_key = reduce(lambda a, b: a + b, [np.random.choice(alphabet) for i in range(16)])

print(f'AES secret key: {aes_key}')

AES secret key: r yeoqkydifpmobd


AES supports multiple operating modes and requires we specify which to use.

We choose the Cipher Block Chaining (CBC) mode provided by the
class of the library. The CBC mode of AES uses randomness for additional security. This requires specifying a random Initialization Vector (IV), also called a . We will use a random string for this as well, just like we did for the key.

In [13]:
aes_initialization_vector = reduce(lambda a, b: a + b, [np.random.choice(alphabet) for i in range(16)])
print(f"AES initialization vector: {aes_initialization_vector}")

AES initialization vector: gkyp xbhg xhdcoe


We can now instantiate an AES cipher on behalf of the sender of the secret message.  

We will then encrypt the plain text to send.

In [14]:
# The encryptor is setup using the key & CBC. In both cases we need to convert the string (utf-8) into bytes

sender_aes_cipher = Cipher(algorithms.AES(bytes(aes_key, 'utf-8')), modes.CBC(bytes(aes_initialization_vector, 'utf-8')))
aes_encryptor = sender_aes_cipher.encryptor()

plaintext=u"nobody can see this message now or never"

# update can add text to encypt in chunks, and then finalize is needed to complete the encryption process

aes_ciphertext = aes_encryptor.update(bytes(plaintext, 'utf-8')) + aes_encryptor.finalize()

# Note the output is a string of bytes

print(f"Encrypted AES ciphertext: {aes_ciphertext}")

ValueError: The length of the provided data is not a multiple of the block length.

Oh, an error. Let's check the length of the plaintext.

In [15]:
# Convierte el texto plano a bytes
plaintext_bytes = bytes(plaintext, 'utf-8')

print(f"\nGiven plaintext: {plaintext}")

# Imprime la longitud del texto plano en bytes
print(f"Plaintext length: {len(plaintext_bytes)} bytes")

# Comprueba si la longitud es múltiplo de 16
if len(plaintext_bytes) % 16 == 0:
    print("Plaintext length is a multiple of the block lenght (16 bytes).")
else:
    print("Plaintext length is a multiple of the block lenght (16 bytes).")



Given plaintext: nobody can see this message now or never
Plaintext length: 40 bytes
Plaintext length is a multiple of the block lenght (16 bytes).


In [16]:
# Contar el número de caracteres ASCII
ascii_count = sum(1 for char in plaintext if ord(char) < 128)

# Imprimir el número de caracteres ASCII
print(f"Número de caracteres ASCII: {ascii_count}")

Número de caracteres ASCII: 40


We can simply generate a more suitable message, with number of ASCII characters (or byte length) proportional to 16. We can also circumvent this issue by automatic padding of the original message.

In [17]:
# The encryptor is setup using the key & CBC. In both cases we need to convert the string (utf-8) into bytes

sender_aes_cipher = Cipher(algorithms.AES(bytes(aes_key, 'utf-8')), modes.CBC(bytes(aes_initialization_vector, 'utf-8')))
aes_encryptor = sender_aes_cipher.encryptor()

# New message with proper number of ASCII characters

plaintext_=u"no one except my dude can never see this message"

# update can add text to encypt in chunks, and then finalize is needed to complete the encryption process

aes_ciphertext_ = aes_encryptor.update(bytes(plaintext_, 'utf-8')) + aes_encryptor.finalize()

# Note the output is a string of bytes

print(f"Encrypted AES ciphertext: {aes_ciphertext_}")

Encrypted AES ciphertext: b"\x1aV\x84\x1a\xb1\x15Z\xa4\x8d4\xf4Y\xa1\xe6'\xd6\x9c)#kW\xc1\x01\xe8e\xc6\xc2\x0e\xff\xbdQ\x8e\x82{g\x9c\xa6\xa2\xb4`\xbf\x84$h\xb7\xac\xe0\xbf"


In [18]:
# We implement padding to make the message length a multiple of 16 bytes

from cryptography.hazmat.primitives import padding

sender_aes_cipher = Cipher(algorithms.AES(bytes(aes_key, 'utf-8')), modes.CBC(bytes(aes_initialization_vector, 'utf-8')))
aes_encryptor = sender_aes_cipher.encryptor()

# Tamaño del bloque de AES es 128 bits (16 bytes)
block_size = 128

# Crea un objeto padder para agregar el relleno PKCS7
padder = padding.PKCS7(block_size).padder()

# Aplica el padding al texto plano
padded_plaintext = padder.update(bytes(plaintext, 'utf-8')) + padder.finalize()

# Encripta el texto con padding
aes_ciphertext = aes_encryptor.update(padded_plaintext) + aes_encryptor.finalize()

# Nota que el resultado sigue siendo un string de bytes
print(f"Encrypted AES ciphertext: {aes_ciphertext}")

Encrypted AES ciphertext: b'`\x8e\xab\x93\xb1\x96n\\\xffkD;w\x8a)\xb0\xe3\xc5\xab2\x0c\x97o\x94yyx\xf2$A\x93\x85\xf2\xf9\xffL\x85<\xa3\x1e\x15\xebLxx\x00\xb7\xaa'


We now show how to decypher the message. Let us try first with the simpler scenario, with a message properly defined to have the adequate bytes length. The logic is similar to the encryption steps.

In [20]:
# Similar setup of AES to what we did for encryption, but this time, for decryption
receiver_aes_cipher = Cipher(algorithms.AES(bytes(aes_key, 'utf-8')), modes.CBC(bytes(aes_initialization_vector, 'utf-8')))
aes_decryptor = receiver_aes_cipher.decryptor()

# Do the decryption
aes_plaintext_bytes = aes_decryptor.update(aes_ciphertext_) + aes_decryptor.finalize()

# convert back to a character string (we assume utf-8)
aes_plaintext_ = aes_plaintext_bytes.decode('utf-8')

print(f"Decrypted AES plaintext: {aes_plaintext_}")

Decrypted AES plaintext: no one except my dude can never see this message


Note that if you try to decrypt the padded message, the plaintext will be likely wrong. 

In [21]:
# Similar setup of AES to what we did for encryption, but this time, for decryption
receiver_aes_cipher = Cipher(algorithms.AES(bytes(aes_key, 'utf-8')), modes.CBC(bytes(aes_initialization_vector, 'utf-8')))
aes_decryptor = receiver_aes_cipher.decryptor()

# Do the decryption
aes_plaintext_bytes = aes_decryptor.update(aes_ciphertext) + aes_decryptor.finalize()

# convert back to a character string (we assume utf-8)
aes_plaintext = aes_plaintext_bytes.decode('utf-8')

print(f"Decrypted AES plaintext: {aes_plaintext}")

Decrypted AES plaintext: nobody can see this message now or nev


We need to remove the padding!

In [23]:
# Create the Cipher object for decryption
cipher = Cipher(algorithms.AES(bytes(aes_key, 'utf-8')), modes.CBC(bytes(aes_initialization_vector, 'utf-8')))
aes_decryptor = cipher.decryptor()

# Decrypt the ciphertext
decrypted_padded_plaintext = aes_decryptor.update(aes_ciphertext) + aes_decryptor.finalize()

# Remove padding
unpadder = padding.PKCS7(algorithms.AES.block_size).unpadder()
unpadded_plaintext = unpadder.update(decrypted_padded_plaintext) + unpadder.finalize()

# Convert bytes back to a string
decrypted_message = unpadded_plaintext.decode('utf-8')

# Print the decrypted message
print(f"Decrypted message: {decrypted_message}")

Decrypted message: nobody can see this message now or never


## ASSYMETRIC KEY CRYPTOGRAPHY (AKC)

### Illustration of the RSA Algorithm

The RSA (RIVEST-SHAMIR-ADLEMAN) algorithm is one of the first public key cryptosystems and is widely used for secure data transmission. It is a versatile cryptosystem: it provides the necessary operations to enable encryption, decryption, digital signatures, and key exchange within a common framework.

In this section, we will illustrate asymmetric key cryptography (AKC) using RSA through simple examples.

As usual, we consider two parties, Alice and Bob, who wish to communicate a message securely using AKC.

#### KEY GENERATION

The key generation step of the RSA algorithm requires of computing the greatest common divisor (gcd) of two integers. This is used to test if two integeres are coprime. In Python, the math module has a method for doing this. We can also define our own function for the same purpose.

In [24]:
import math as mt

# Example function to compute the gcd of two numbers

def gcd(a,b):
    if b==0:
        return a
    else:
        return gcd(b,a%b)
    
# let's check some examples using our function and the math module

n1=gcd(50,10)
n2=gcd(99,33)
n3=gcd(59,9)

m1=mt.gcd(50,10)
m2=mt.gcd(99,33)
m3=mt.gcd(59,9)

# Confirm they are the same
assert(n1==m1)
assert(n2==m2)
assert(n3==m3)

# They are - print out the values for explanation
print("gcd(50,10) =", m1)
print("gcd(99,1033) =", m2)
print("gcd(59,9) =", m3)

gcd(50,10) = 10
gcd(99,1033) = 33
gcd(59,9) = 1


Let's start with the actual key generation step of the RSA! We have to choose two prime numbers which shall be kept secret. Ideally, the prime numbers must have different lengths.

PD: We might need to check if the choices are really prime numbers. We can do this by the Sympy built-in method.

In [25]:
%pip install sympy

Note: you may need to restart the kernel to use updated packages.


In [27]:
from sympy import isprime

# Choose two numbers and check if they are prime

p = 17
q = 113

print(f'The secret prime numbers are {p, isprime(p)} and {q, isprime(q)}')

The secret prime numbers are (17, True) and (113, True)


The next step is to compute the modulus (n) and the totient ($\phi$(n)) of the chosen prime numbers. 

The modulus n will be shared publicly. The Totient function will be needed for the modular multiplicative inverse used to determine the keys, and will be kept secret. It is also typically discarded after key generation.  

In [30]:
n = p * q
phi = (p - 1) * (q - 1)

print(f'The public modulus n = {n} and the Euler totient function [phi(n)] = {phi}')

The public modulus n = 1921 and the Euler totient function [phi(n)] = 1792


We can now compute the public and private RSA keys. We will later specify them as a tuple of integers, the first entry being different, while the second entry will always be the modulus n.

The first entry in the public key can be any integer $e$ greater than 1 that is coprime to $\phi$(n)

The private key requires an integer $d$, which is the multiplicative inverse of $e$, satisfying the congruence relation $d * e \equiv 1\,\mathrm{mod}\phi(n)$. The suitable $d$ might be found by looping over the positive integers. 

PD: In real-life settings, people use the computationally efficient extended Euclidean algorithm.

In [35]:
# Choose an integer e such that e and φ(n) are coprime
e = 2
while (e < phi):
    if (mt.gcd(e, phi)==1):
        break
    else:
        e += 1
        
# Compute a value for d such that (d * e) % φ(n) = 1
d = 1
while(True):
    if((d*e) % phi == 1):
        break
    else:
        d += 1
        
# We form the tuples for the public and private keys

pubkey = (e,n)
privkey = (d,n)
            
print("Private Key (d,n):", privkey)
print("Public Key (e,n):", pubkey)

Private Key (d,n): (1195, 1921)
Public Key (e,n): (3, 1921)


In RSA, encryption and decryption use the modular exponentiation algorithm. The complementarity between the public and private keys allows for encryption and decryption of a given message. 

Let us illustrate both steps on an integer message.

PD: We use here small numbers for illustration. In practice, RSA requires of the usage of very large integers. In 2048-bit RSA we shall use a modulus $n$ that is 2048 bits long. This is crucial for practical security of the protocol.

In [36]:
def encrypt(plaintext):
    return (plaintext ** e) % n

def decrypt(ciphertext):
    return (ciphertext ** d) % n

# Define a simple message to encode (an integer)

msg = 9

# Encrypt & Decrypt the message

enc_msg = encrypt(msg)
dec_msg = decrypt(enc_msg)

print("Original Message:",msg)
print("Encrypted Message:",enc_msg)
print("Decrypted Message:",dec_msg)

Original Message: 9
Encrypted Message: 729
Decrypted Message: 9


#### SYMMETRIC KEY EXCHANGE

AKC enables secure communication between Alice and Bob by establishing a shared secret, which can be used, for instance, as the secret key for symmetric encryption of a given message.

We now illustrate the two ways Alice and Bob have to share this secret key. These are the Padding-based Key Exchange or the Key Encapsulation Mechanism.

#### Padding-based key exchange

This is a non-interactive key exchange. In Python, we can implement it by means of the cryptography module. Alice generates a random secret key, which then shall be transmitted to Bob.

In [40]:
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.fernet import Fernet
from cryptography.hazmat.primitives import hashes

# Alice generates a symmetric key

a_symmetric_key = Fernet.generate_key()
print(f"\nSymmetric key generated by Alice: {a_symmetric_key}")

# Bob generates a 2048-bit RSA key pair

bob_private_key = rsa.generate_private_key(public_exponent=65537, key_size=2048)
bob_public_key = bob_private_key.public_key()
print(f"\nPublic key broadcast by Bob: {bob_public_key}")
print(f"Public numbers in Bobs' public key: {bob_public_key.public_numbers()}")


Symmetric key generated by Alice: b'X01oYSsuaNocSSx0MuEymIwuc8P3y1KvtL97pP5ReoU='

Public key broadcast by Bob: <cryptography.hazmat.bindings._rust.openssl.rsa.RSAPublicKey object at 0x11ab22a90>
Public numbers in Bobs' public key: <RSAPublicNumbers(e=65537, n=22775269966844386908382650347145375119290038890115977323139224851673190694800644504975986369545861992307757396422418500912961982254771317084169403811846482060382979780090095156335213240585333262514313215070913787726898343224406452227807116093685006052592600729730381252970106489795982009508343362312494437638315161214838794654628060607448546450192009741527598095760878911741968177388513395703362622175656103555062004596485282278441231829839511504331171999963905848170023610159721921652275347012723295976506282150974252974158754538476562718926324638260686467553609002910130339746101541252392830219217485091301762880389)>


Let us now assume that Alice has received the public key broadcast by bob. Alice's symmetric key above will now be encrypted using Bob's public key to produce a ciphertext. Realistically, Alice has to use additional padding methods such as OAEP to enforce semantic security.

In [41]:
# Encryption
ciphertext = bob_public_key.encrypt(
   a_symmetric_key,
   padding.OAEP(
       mgf=padding.MGF1(algorithm=hashes.SHA256()),
       algorithm=hashes.SHA256(),
       label=None
   )
)

print("Alice generated Ciphertext:", ciphertext)

Alice generated Ciphertext: b'\\\x8d\xa1\xa8\x81\xbc\x9b\xd4\x9d\x89\x85\x84#\x1e@3\x1e{\xd7\xb1n9!F\xee\xa0\xd0pq\xf8B\x98\xf8\x0c\xb2\xcf\xda\xd1\x0b\xa86)\xe5]\x14\xe4\xa8\xfd5\xb8\xaa8#\xed\x04\xde\xd5\xadK\x86\x1c\x04y$\xb7.)ui\x07)\xe1h\x17\x02U\x81z?\xf9\x0f-\xc0\xd8\xb1\xbc\xe8\x1aN;a\xfb\xc6z\xffJ\x81\xec\xdd\x0c\\+U\x118\x17\x80n\xea\x12\x01;N\xb1\xf7\xb9"\xd2\x07p\x1f\x96\x7f&B\xaa\xdb"\x18\x92G\xe7B\xa4\x0es\xa6\xfb\xbdW8\x0c\x98\xf9K\xdf\xfa;\x9dB\x05\xc0G\x1e\xc2\x91{\t\xed_=P\x99ZGC\t\x94x\xcd\xdd \x1b\tV\xe5#61\xdb:\xca\xfdH\xfa\x96g\xef0]\xb7\x83\xe6\xb1\x0f+\x08\x8c\xa3\xb9\x9e\x00\xfeo\xd4\xbf#\x9a\xd0!\x9b\xb7\x0e\x05\x1a\x16\xb2>)\x82\x17#(L\x89*\x14\x95C\x8e\x7f\xb2{\x11w\x99\xb1]\xa0\xfa\xd1\xc7f\x87\xba\xcb\xd0\xbc\xe6Z\\\x14\xefu\x89,'


Now, Alice sends the ciphertext over an open channel, trusing that only Bob can decrypt it by using the corresponding private key. We assume the communication step and illustrate Bob work.

In [42]:
# Bob decrypts ciphertext to access the symmetric key
decrypted_symmetric_key = bob_private_key.decrypt(
   ciphertext,
   padding.OAEP(
       mgf=padding.MGF1(algorithm=hashes.SHA256()),
       algorithm=hashes.SHA256(),
       label=None
   )
)

print("Decrypted key:", decrypted_symmetric_key)
assert decrypted_symmetric_key == a_symmetric_key

Decrypted key: b'X01oYSsuaNocSSx0MuEymIwuc8P3y1KvtL97pP5ReoU='


Alice and Bob have now access to the secret symmetric key, which they can use for SKC. 

#### KEY ENCAPSULATION MECHANISM (KEM)

We can also simulate a Key Encapsulation Mechanism (KEM) whereby a sufficiently long random secret message is securely exchanged and subsequently converted into a shared-secret of the appropriate length using a KDF.

Once again Alice and Bob want to establish a shared secret non-interactively and Alice is the party that decides which secret to use.

Bob can generate his RSA key pair and broadcast his public key.

In [43]:
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives import hashes, serialization
from os import urandom

# Bob's RSA key pair

private_key_Bob = rsa.generate_private_key(public_exponent=65537, key_size=2048)
public_key_Bob = private_key_Bob.public_key()

print("Bob's private and public keys created")

Bob's private and public keys created


Alice first generates a long random secret from which the shared secret will be eventually derived. In a pure KEM, the long secret will be a random element from the algebraic structure underlying the cryptosystem. 

PD: pure KEM does not require additional padding but in this example we are only simulating a KEM using RSA and the cryptography library requires the use of padding when encrypting with RSA. So we will use a somewhat shorter long secret which nevertheless is much longer than a standard 256-bit AES key.

In [45]:
# Alice generates a 1280 bit long secret (160 byte). Note that this is smaller than 2048-bit RSA, hence the need for additional padding.

Alice_long_secret = urandom(160)
print("Alice's secret created")

# Alice encrypts the long secret using Bob's public key and the encrpypted secret is communicated to Bob

Alice_encrypted_secret = public_key_Bob.encrypt(
    Alice_long_secret,
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None
    )
)
print("\nAlice's secret encrypted")

Alice's secret created

Alice's secret encrypted


Bob receives the encrypted message. He has to decrypt the secret message by using his private key.

In [46]:
Bob_decrypted_secret = private_key_Bob.decrypt(
    Alice_encrypted_secret,
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None
    )
)

#Chek if the decrypted secret matches the original secret

assert Alice_long_secret == Bob_decrypted_secret, "Secrets do not match!"

# if we get here they match, if not code stops above
print("Secrets match")

Secrets match


The final step of a KEM is that both Alice and Bob have to apply an agreed upon Key Derivation Function (KDF) on the long secret to derive the symmetric key. 

This process involves a hashing protocol and the use of a random salt. This ensures uniqueness and unpredictability of the shared symmetric key in case the long secret is ever reused. However, the latter is obviously not recommended in real applications. Importantly, the salt itself does not have to be secret. In this example, the salt is randomly generated and then broadcast to bob alongside the encrypted long secret. 

In [47]:
# Define a key derivation function using HKDF

def key_derivation_function(secret, salt):
    hkdf = HKDF(
        algorithm=hashes.SHA256(),
        length=32,  # Desired key length
        salt=salt,
        info=None,
        backend=None
    )
    return hkdf.derive(secret)

# Random salt accessible to both Alice and Bob, for proper seasoning of the secret!

common_salt = urandom(16)  

symmetric_key_Alice = key_derivation_function(Alice_long_secret, common_salt)
symmetric_key_Bob = key_derivation_function(Bob_decrypted_secret, common_salt)

assert symmetric_key_Alice == symmetric_key_Bob, "Derived keys do not match!"
print(f"A symmetric key of length {len(symmetric_key_Alice)*8} bits was successfully derived by both Alice and Bob!")

A symmetric key of length 256 bits was successfully derived by both Alice and Bob!


#### DIGITAL SIGNATURES WITH RSA

We extend the above confidential communication scenario with Alice and Bob to include validation with the help of a digital signature.

Alice will confidentially send a message encapsulating a symmetric key to Bob, but she will also digitally sign the message so that Bob, upon receiving the message, can verify that it was Alice who originated it and that the contents of the message were not tampered with during transmission.

More generally, it is desirable to enable validation without compromising confidentiality whereby any interested party is able to verify the integrity, authenticity and establish non-repudiation with respect to a given communication, even if that party does not have access to the actual plain text message.

We will consider this general setting which then involves the following steps:

- First, both Bob and Alice make their public keys available over an open channel.

- Then, Alice encrypts the plain text using Bob's public key, creating a ciphertext.

- Next, Alice creates a hash of the ciphertext with a hash function and further encrypts the resulting hashed ciphertext using her private key. This encrypted hash is the signature.

- Then, Alice then transmits both the ciphertext and the signature over an open channel.

- Then, Bob uses Alice's public key to decrypt the signature, revealing the hashed ciphertext.

- Next, since Bob also has access to the ciphertext itself, he uses the same hash function used by Alice to recreate a second instance of the hashed ciphertext. If the latter matches the one obtained by decrypting Alice's signature, then the message is validated, even though the ciphertext itself has not yet been decrypted.

- Finally, Bob, having validated the message, decrypts the ciphertext using his own private key.

In [52]:
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.hazmat.primitives.asymmetric import padding, rsa
from cryptography.hazmat.primitives.asymmetric.utils import Prehashed

# Generate keys for Bob

bob_private_key = rsa.generate_private_key(public_exponent=65537, key_size=2048)
bob_public_key = bob_private_key.public_key()

# Generate keys for Alice

alice_private_key = rsa.generate_private_key(public_exponent=65537, key_size=2048)
alice_public_key = alice_private_key.public_key()

print("Private and Public keys generated for Bob and Alice.")

# Alice encrypts the message using Bob's public key

ciphertext = bob_public_key.encrypt(
    a_symmetric_key,
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None
    )
)

print("\nciphertext of symmetric key: ",ciphertext)

Private and Public keys generated for Bob and Alice.

ciphertext of symmetric key:  b'z/\'\xf3M\xf7\xb2\x87\xcb<\\\x90D\xfb\xfa\xba\x7f\xb2\xa5\xed1\x8d\x12"\xa5\xb26=\x9b\xfb2\x1d\x86\x07\x91P\x03Q\xc9\x1a\xa4\xb7fV\xa7\x1av\x9c\xd2Z(\xf2\x90\x97\x0b>J\x86\x04\x89\xc6\xfc\xb3`j#\xa1\n\x94!\x13\xd7L \x18\xc1\xf5AN\xe4B\xa2d,\x11\xeb\xf5\xc5g\n\xdd\xa0r\xf4&s!8\x9d\xfbt\xd3\xb0[t\x9d\x10\x15\x93\xdd\x9c\xe0^b\xc0\xbbGk\x81\x83\xa7\xae\xaeP\x7f!\xeb\x9d\xb9xgT:\x14J\x00\xf0u\x82\xeb\x19\xaf\xb5\x1e{\xea\xc5\xfa\xa4\xbd>\xb3 9\xc8{I\xd7\x87\x8e\x91e\xe8\xe2\xfc+\xdb\x93\x11s\xcc\x7f_\xde\xe6sDx\x89\x00\x8f\xb4*\x152\xfd#dK\xa6.\xd3$\x8d\xe0\x8aY\x92\xe4\x92\xd4\xd6(\xac`\xdd\x88\x98,\xae\xb2\x11\x08\xad\xec\xebB\xf6kq\xafW\x8ce\xe3\n\xf4\x98\xcc\xa1\xc1\x85\xf6\xc1\x0b\xa2\xc1\xdd\xe2;\xf28\xcc\xe5\x9a\xa2\x9b4f\xfe\x18\x9f\xb7\xdf\xa2\xfb'


Until here, nothing new. We introduce now the intermediate step of attaching a digital signature before broadcasting the ciphertext. 

The digital signature will be done by creating a hash of the ciphertext and encrypting this has using Alice's private key. This is Alice's signature.

In [53]:
# Alice hases the ciphertext, i.e. obtains the digest of the ciphertext.

digest = hashes.Hash(hashes.SHA256())
digest.update(ciphertext)
hash_to_sign = digest.finalize()

# Alice signs the ciphertext using her private key

signature = alice_private_key.sign(
    hash_to_sign,
    padding.PSS(
        mgf=padding.MGF1(hashes.SHA256()),
        salt_length=padding.PSS.MAX_LENGTH
    ),
    Prehashed(hashes.SHA256())
)

print("signature: ", signature)

signature:  b't\x0b\x138\x1e\x10T.qp\xec\x0b\x08H\x00\xdd\x8e\x97\xb3\xba7\x9dyf\x01\xa4A\xaa\xc1is\'i{\xbb\\\xc6\xe4\x99\xc8\xae\x02\x0f\x17?Z\xc1\x17\xa6P|\x9a]nr\xddN\xb4\x90<\xe1Q\x8b\xf8\x91[O;\x03%\r{\xb0\xb5\xb1w\x00o_\xc7JX\x19fL4jl\x8fg\xa1s\x12f\x9a\xcb\xfd\xc6&N\x8d\x11,l\x15\xd2u`M@\xe6W\xea\x80\x17\x14\x9e\xf9\xd2\x04\xc4l4\x7fJt\xf0s\xec^^/Q{>\x8ac\x01\xc6\xed\xc8E\x9c\xa4/\x97\n\x99,1GE\xc8F\x9f\xe7P*\xebZ\xe6lg\xfd\xf6\xa0\xf3\x9e\x93\xf1"\xdfp-|$\xcfd\xf2LR\xe0\xa6\xb2\xcb\xe6\x8aB{\x99\x93\x96s(S+\xf1\xa9E\x14\'\x1fAo\x01\x1a\xd4GW)/\x91o\xd0\xe4\xbb\xe3\xa9\x94V\t\xb6\x1f\xc9\x14\x9eZ\xe8][mr\xf5\xd2\xd6@\xf1\xab\xe6\xe1\t\xd6\xcdN{\xee\xfa\xf69}\x9a\x19*\xa4;0'


It is now when the ciphertext is broadcasted together with the digital signature. 

Bob's receive the message. He can now first verify the integrity and authenticity of the ciphertext. 

PD: Hasing algorithms must be the same!

In [54]:
# Bob receives the ciphertext and signature

received_ciphertext = ciphertext
received_signature = signature

print("Receiving ciphertext and signature .....")

#Bob creates a hash of the ciphertext using the same algorithm used by Alice

digest = hashes.Hash(hashes.SHA256())
digest.update(received_ciphertext)
hash_to_verify = digest.finalize()

print("\nhash to verify: ", hash_to_verify)

# Bob verifies the signature using Alice's public key
try:
    alice_public_key.verify(
        received_signature,
        hash_to_verify,
        padding.PSS(
            mgf=padding.MGF1(hashes.SHA256()),
            salt_length=padding.PSS.MAX_LENGTH
        ),
        Prehashed(hashes.SHA256())
    )
    print("The signature is valid.")
except:
    print("The signature is not valid.")

Receiving ciphertext and signature .....

hash to verify:  b'{\xf8\x07\xd54\x08\x13qZm\x10\x9c\x08\x1a\x97\x7fmZ\xf7\xd3f\x15\xe4\xb0Y\x8b\x1b\xbfcV\xeb\xb9'
The signature is valid.


Now that Bob has verified the integrity and authenticity of the received ciphertext, Bob can trust to decrypt the secret message using his private key. Recall that Alice has created the ciphertext using Bob's public key.

In [57]:
# Bob decrypts the message using his private key

decrypted_message = bob_private_key.decrypt(
    received_ciphertext,
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None
    )
)

print("Decrypted message:", decrypted_message.decode())

# Check if the decrypted message is the same as the original symmetric key (nothing happens if True)
assert decrypted_message == a_symmetric_key

Decrypted message: X01oYSsuaNocSSx0MuEymIwuc8P3y1KvtL97pP5ReoU=


In this scenario, any party can verify that Alice sent the ciphertext. Everyone has access to the Alice's public key, the ciphertext and the digital signature. In addtion, Alice cannot deny having sent the ciphertext later, because the signature can be decrypted to a meaningful has using only her public key (wich is only shared upon Alice actively communicating the ciphertext). This is the foundation of the 'non-repudiation' mechanism enclosed in Digital Signature Algorithms.

### Breaking RSA

The utility and security of the RSA algorithm is essentially based on the prime factorization. This can be done classically by several algorithms, and also quantically by using the Shor's Algorithm (or any updated version of it). While we are not getting into theoretical details here, let us mention that even though the 2048-RSA algorithm is at present considered secure against classical systems attacks, meaning that any classical algorithm is good enough for factoring 2048-bits integers (i.e. of order $\mathcal{O}(10^{617})$), this will not be the case when quantum computers available. 

Let us illustrate an example of finding a private key given only the public key. This will use brute-force classical computation, but shows how Shor's algorithm could be used - including large keys.

In this example, supose we have a public key $(5, 247)$. We will attempt to recover the private key.

In [58]:
n = 247 # The modulus of the public key
e = 5 # The public key number (exponent)
a = 6 # The number coprime to n

assert gcd(a, n) == 1, "a and n are not coprime"
print(f'Checked {n} and {a} are coprime')

# In this example, almost any number will work as the coprime.

Checked 247 and 6 are coprime


In [59]:
# We try to find the period r such that a^r = 1 mod n. Here we do it classically.

r=0
rem = 100
while(rem != 1):
    r += 1
    rem = (a**r) % n
    
print(f'period r is: {r}')
assert a**r % n == 1

print(f"Checked {a}^{r} mod {n} is 1")


period r is: 36
Checked 6^36 mod 247 is 1


In [60]:
# The period r is even. Hence, we compute the following numbers

# explicitly use as integer
f1 = int ( a**(r/2) - 1)
f2 = int ( a**(r/2) + 1)

print(f"f1 = {f1}")
print(f"f2 = {f2}")

f1 = 101559956668415
f2 = 101559956668417


In [61]:
# Now we find the GCD of either of those f1 or f2 with n. 

q_found = gcd(f1, n)
print(f'One possible prime factor of n ({n}) is: {q_found}')

# explicit int (to avoid floating point)
p_found = int ( n/q_found )
print(f'The second prime factor of n ({n}) is: {p_found}')

assert n == p_found * q_found


One possible prime factor of n (247) is: 19
The second prime factor of n (247) is: 13


In [62]:
# We compute the totient of the found q and p factors

phi_found = ( p_found -1 ) * ( q_found - 1 ) 
print(f'The totient is: {phi_found}')

#Recover the private key number d_found by satisfying (d_found * e) % phi_found = 1

d_found = 1
while(True):
    if((d_found*e) % phi_found == 1):
        break
    else:
        d_found += 1
print("Private Key number:",d_found)


The totient is: 216
Private Key number: 173


In the above scheme, the crucial step is the second one: the period-finding operation. For this operation, the Shor's algorithm uses two fundamental quantum primitives, i.e. the quantum fourier transform and quantum phase estimation. 